Workflow automation tools
=========================

Running several simulations with PyFMI
--------------------------------------

This page shows how to compile and run a Modelica model in Python, using the FMI standard.

The [PyFMI](https://pypi.python.org/pypi/PyFMI) package is required, which is only compatible with Python 2.7. The easiest way for a functional PyFMI framework is to install the [JModelica platform](http://jmodelica.org/) and to run Python scripts from its IPython or pylab console. The JModelica installer includes a Python 2.7 distribution, which you can use in addition to an eventual other one.

This exercise requires the [Annex60 Modelica library](https://github.com/iea-annex60/modelica-annex60). Before running the code, the `ppath` variable below should point to the location of this library on your drive.

In [ ]:
import numpy as np

#==============================================================================
# Compile the FMU from the .mo file
#==============================================================================
from pymodelica import compile_fmu
ppath = 'C:\\path_to_the_modelica_library_on_your_drive'
model_name = 'Annex60.Controls.Continuous.Examples.PIDHysteresis'
fmu1 = compile_fmu(model_name, ppath)

#==============================================================================
# Loading the FMU
#==============================================================================
# Without JModelica, you can skip the part above and load a .fmu file that
# has been generated by another simulator

from pyfmi import load_fmu
PID = load_fmu('Buildings_Controls_Continuous_Examples_PIDHysteresis.fmu')

# Choice of the time discretisation
tStart = 0
tStop = 3600*24

#==============================================================================
# First simulation to initialize the FMU
#==============================================================================

# Simulation options: choice of time step, and no automatic initialization
opt = PID.simulate_options()
opt['initialize'] = True
# Read the value of the initial PID gain
k_init = 3000
PID.set('gain.k', k_init)
print("Initial PID gain: %0.1f ") % k_init
# Simulate and retrieve the temperature profile
PID_res = PID.simulate(tStart, tStop, options = opt)
t = PID_res['time']
T = PID_res['temSen.T']
T_set  = PID.get('TSet.k')* np.ones(len(t))

# Plot the temperature profile of the initial simulation compared to the set point
import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex=True)
rc('font', family='serif', size = 14)

fig = plt.figure()
plt.plot(t/3600, T-273.15, '-k', linewidth = 1.5, label = '$T$')
plt.plot(t/3600, T_set-273.15, '--r', linewidth = 1.5, label = '$T_{set}$')
plt.fill_between(t/3600, T-273.15, T_set-273.15, color='grey', alpha='0.5')
plt.xlabel('Time (h)')
plt.ylabel('Temperature (C)')
plt.legend()
plt.savefig('JModelica_Ex2_plot.png')

#==============================================================================
# Simulation method
#==============================================================================
# Simulate the temperature profile as a function of the PID gain
# then calculates the average quadratic deviation from the temperature set point

from scipy.interpolate import interp1d
t_interp = np.linspace(tStart, tStop)

def simulate_case_gain(k):
    
    PID.reset()

    # How to modify a parameter value
    PID.set('gain.k', k)
    
    # Simulation of the 'boiler' object
    PID_res = PID.simulate(tStart, tStop)
    
    # Read results
    t = PID_res['time']
    T = PID_res['temSen.T']
    
    # Interpolate the temperature on a time scale with evenly spaced points
    T_interp = interp1d(t,T)(t_interp)
    
    return np.mean( (T_interp-PID.get('TSet.k'))**2 )

#==============================================================================
# Main algorithm
#==============================================================================

# List of PID gain values to test
k_list = np.linspace(1000, 3000, num = 21)
# List of mean quadratic temperature errors
e_list = [simulate_case_gain(_) for _ in k_list]

fig = plt.figure()
plt.plot(k_list, e_list, '-ok')
plt.xlabel('PID gain')
plt.ylabel('$(T-T_{set})^2$')
plt.savefig('JModelica_Ex2_plot2.png')
